# RAG Attack - Partie 2: Agentic RAG avec LangGraph

## Introduction aux Systèmes Agentiques

Bienvenue dans cette formation sur les systèmes RAG agentiques ! Après avoir vu les bases du RAG (embeddings, vector search) dans la partie 1, nous allons maintenant explorer comment rendre ces systèmes intelligents et autonomes.

### Qu'est-ce qu'un Agent ?

Un **agent** est un système qui peut :
- **Raisonner** sur une tâche
- **Planifier** une séquence d'actions
- **Utiliser des outils** pour accomplir ses objectifs
- **S'adapter** en fonction des résultats obtenus

### Pourquoi LangGraph ?

LangGraph nous permet de créer des workflows complexes sous forme de graphes d'états, parfait pour orchestrer des agents intelligents.

## Setup et Configuration

Commençons par installer les dépendances nécessaires et configurer notre environnement Azure.

In [ ]:
# Installation des dépendances
!pip install langgraph langchain langchain-openai azure-search-documents azure-identity pyodbc pandas requests duckduckgo-search -q

In [ ]:
# Import des bibliothèques nécessaires
import sys
import os
from pathlib import Path

# Ajouter le package rag_attack au path
sys.path.insert(0, str(Path.cwd()))

# Imports de base
import json
from typing import List, Dict, Any
from IPython.display import display, Markdown, HTML
import warnings
warnings.filterwarnings('ignore')

print("✅ Environnement configuré avec succès !")

In [ ]:
# Chargement de la configuration Azure
from rag_attack.utils.config import load_azure_config, get_openai_client

try:
    config = load_azure_config()
    print("✅ Configuration Azure chargée avec succès !")
    print(f"   - Azure OpenAI Endpoint: {config['openai_endpoint']}")
    print(f"   - Azure Search Endpoint: {config['search_endpoint']}")
    print(f"   - SQL Server: {config['sql_server']}")
except Exception as e:
    print(f"❌ Erreur lors du chargement de la configuration: {e}")
    print("Assurez-vous d'avoir exécuté 'make setup-all' dans scai_onepoint_rag/")

## Chapitre 1: Agent Simple avec Outils RAG

### 1.1 Concept de Base

Un agent simple suit ce cycle :
1. **Recevoir** une question
2. **Décider** si un outil est nécessaire
3. **Utiliser** le bon outil
4. **Formuler** une réponse

### 1.2 Implémentation

In [ ]:
# Import des outils RAG
from rag_attack.tools import azure_search_tool, azure_vector_search_tool
from rag_attack.agents import SimpleToolAgent

# Création d'un agent simple avec l'outil de recherche Azure
simple_agent = SimpleToolAgent(
    tools=[azure_search_tool],
    system_prompt="""Tu es un assistant expert en recherche d'information sur VéloCorp.
    Utilise l'outil de recherche Azure pour trouver des informations pertinentes.
    Réponds en français de manière claire et structurée."""
)

print("✅ Agent simple créé avec l'outil de recherche Azure")

In [ ]:
# Test de l'agent simple
question = "Quels sont les modèles de vélos électriques proposés par VéloCorp ?"

print(f"❓ Question: {question}\n")
print("🤖 Réponse de l'agent:\n")
print("-" * 50)

response = simple_agent.invoke(question)
display(Markdown(response))

### 1.3 Visualisation du Flux de l'Agent

Voyons comment l'agent traite la requête étape par étape :

In [ ]:
# Streaming pour voir le processus en temps réel
def visualize_agent_stream(agent, question):
    """Visualise le flux de traitement de l'agent"""
    print(f"📝 Question: {question}\n")
    
    for i, chunk in enumerate(agent.stream(question)):
        print(f"\n🔄 Étape {i+1}:")
        
        # Afficher le type de nœud
        for node, data in chunk.items():
            print(f"   📍 Nœud: {node}")
            
            if 'messages' in data:
                last_msg = data['messages'][-1]
                
                # Afficher le type de message
                if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                    print(f"   🔧 Appel d'outil détecté: {last_msg.tool_calls[0]['name']}")
                elif hasattr(last_msg, 'content'):
                    content_preview = str(last_msg.content)[:200]
                    print(f"   💬 Contenu: {content_preview}...")

# Test avec visualisation
visualize_agent_stream(simple_agent, "Quelle est la politique de garantie de VéloCorp ?")

## Chapitre 2: Agent ReAct (Reasoning and Acting)

### 2.1 Le Framework ReAct

ReAct est un paradigme puissant qui alterne entre :
- **Thought** (Réflexion) : L'agent raisonne sur ce qu'il doit faire
- **Action** : L'agent exécute une action (utilise un outil)
- **Observation** : L'agent observe le résultat

Cette approche rend le processus de décision **transparent** et **débogable**.

In [ ]:
# Import de l'agent ReAct
from rag_attack.agents import ReActAgent
from rag_attack.tools import azure_search_tool, sql_query_tool, get_database_schema

# Création d'un agent ReAct avec plusieurs outils
react_agent = ReActAgent(
    tools=[
        azure_search_tool,
        sql_query_tool,
        get_database_schema
    ],
    max_iterations=5
)

print("✅ Agent ReAct créé avec plusieurs outils")

In [ ]:
# Test de l'agent ReAct avec une question complexe
complex_question = """Trouve-moi les informations sur les vélos électriques de VéloCorp 
et vérifie dans la base de données s'il y a des commandes récentes pour ces modèles."""

print(f"❓ Question complexe: {complex_question}\n")
print("="*60)

# Obtenir la trace complète du raisonnement
trace = react_agent.get_reasoning_trace(complex_question)

# Afficher le processus de raisonnement
print("\n🧠 PROCESSUS DE RAISONNEMENT:\n")
for i, (thought, obs) in enumerate(zip(trace['reasoning'], trace['observations']), 1):
    print(f"\n{'='*60}")
    print(f"Étape {i}:")
    print(f"\n💭 Réflexion: {thought}")
    print(f"\n👁️ Observation: {obs}")

print(f"\n{'='*60}")
print(f"\n✅ RÉPONSE FINALE:\n{trace['final_answer']}")

### 2.2 Comparaison: Agent Simple vs ReAct

Comparons les deux approches sur la même question :

In [ ]:
# Question de test
test_question = "Quel est le chiffre d'affaires de VéloCorp et combien de clients ont-ils ?"

print("📊 COMPARAISON DES APPROCHES\n")
print("="*60)

# Test avec l'agent simple
print("\n1️⃣ AGENT SIMPLE:")
print("-"*40)
simple_response = simple_agent.invoke(test_question)
print(simple_response[:500] + "..." if len(simple_response) > 500 else simple_response)

# Test avec l'agent ReAct
print("\n2️⃣ AGENT REACT:")
print("-"*40)
react_response = react_agent.invoke(test_question)
print(react_response[:500] + "..." if len(react_response) > 500 else react_response)

print("\n" + "="*60)
print("\n💡 Observation: L'agent ReAct fournit un raisonnement plus structuré !")

## Chapitre 3: Agent Multi-Outils

### 3.1 Intégration de Multiples Sources de Données

Un agent vraiment utile doit pouvoir orchestrer plusieurs outils :
- **Recherche documentaire** (Azure Search)
- **Base de données** (SQL)
- **APIs externes** (CRM, Weather, etc.)
- **Web** (recherche internet)

In [ ]:
# Import de tous les outils disponibles
from rag_attack.tools import (
    azure_search_tool,
    azure_vector_search_tool,
    sql_query_tool,
    sql_table_info,
    get_database_schema,
    crm_opportunities_tool,
    weather_api_tool,
    web_search_tool
)

# Création d'un super-agent avec tous les outils
from rag_attack.agents import SimpleToolAgent

super_agent = SimpleToolAgent(
    tools=[
        azure_search_tool,
        sql_query_tool,
        sql_table_info,
        crm_opportunities_tool,
        weather_api_tool,
        web_search_tool
    ],
    system_prompt="""Tu es un assistant expert multi-domaines pour VéloCorp.
    Tu peux:
    - Rechercher dans la documentation interne
    - Interroger la base de données
    - Accéder au CRM pour les opportunités commerciales
    - Obtenir la météo (utile pour planifier des événements vélo)
    - Rechercher sur internet pour des informations complémentaires
    
    Utilise intelligemment ces outils pour fournir des réponses complètes et pertinentes."""
)

print("✅ Super-agent créé avec accès à:")
for tool in super_agent.tools:
    print(f"   - {tool.name}")

In [ ]:
# Test du super-agent avec une requête complexe nécessitant plusieurs outils
multi_tool_question = """Je prépare une présentation pour les investisseurs. 
J'ai besoin de:
1. Les dernières opportunités commerciales du CRM
2. Les statistiques de ventes depuis la base de données
3. La météo à Paris pour notre événement de lancement la semaine prochaine
"""

print(f"📋 Requête multi-outils:\n{multi_tool_question}\n")
print("="*60)
print("\n🤖 Traitement en cours...\n")

response = super_agent.invoke(multi_tool_question)
display(Markdown(response))

### 3.2 Cas d'Usage: Analyse Business Complète

In [ ]:
# Fonction d'aide pour une analyse business complète
def business_analysis(company_context: str, agent: SimpleToolAgent):
    """Effectue une analyse business complète"""
    
    analyses = {
        "📊 Analyse des Ventes": "Quelles sont les tendances de ventes récentes dans la base de données ?",
        "🎯 Opportunités CRM": "Quelles sont les opportunités commerciales à fort potentiel ?",
        "📚 Documentation Produit": "Quels sont nos principaux produits et leurs caractéristiques ?",
        "🌍 Analyse Marché": "Recherche sur internet les dernières tendances du marché du vélo électrique",
    }
    
    results = {}
    
    for title, question in analyses.items():
        print(f"\n{title}")
        print("-" * 40)
        
        full_question = f"{company_context} {question}"
        response = agent.invoke(full_question)
        
        results[title] = response
        print(response[:300] + "..." if len(response) > 300 else response)
    
    return results

# Exécution de l'analyse
context = "Pour VéloCorp, entreprise de vélos électriques premium,"
print("🔍 ANALYSE BUSINESS COMPLÈTE")
print("="*60)

analysis_results = business_analysis(context, super_agent)

## Chapitre 4: Architecture Planner

### 4.1 Le Concept de Planification Hiérarchique

Pour des tâches vraiment complexes, nous avons besoin d'un **planner** qui peut :
1. **Décomposer** une tâche complexe en sous-tâches
2. **Gérer les dépendances** entre les tâches
3. **Orchestrer l'exécution** dans le bon ordre
4. **S'adapter** si quelque chose échoue

In [ ]:
# Import du planner hiérarchique
from rag_attack.planners import HierarchicalPlanner

# Création du planner avec tous les outils
planner = HierarchicalPlanner(
    tools=[
        azure_search_tool,
        sql_query_tool,
        get_database_schema,
        crm_opportunities_tool,
        web_search_tool
    ],
    max_steps=8
)

print("✅ Planner hiérarchique créé")

In [ ]:
# Test du planner avec un objectif complexe
complex_objective = """Prépare un rapport complet pour le conseil d'administration incluant:
1. Analyse de la performance commerciale actuelle
2. Étude de marché sur les tendances du vélo électrique
3. Recommandations stratégiques basées sur les données
"""

print(f"🎯 Objectif complexe:\n{complex_objective}\n")
print("="*60)
print("\n⚙️ Exécution du plan...\n")

# Exécution avec le planner
result = planner.execute(complex_objective)

# Affichage du plan
print("\n📋 PLAN GÉNÉRÉ:")
print("-" * 40)
for step in result['plan']:
    status_emoji = "✅" if step['status'] == "completed" else "⏳"
    print(f"{status_emoji} Étape {step['step_id']}: {step['description']}")
    if step['tool']:
        print(f"   🔧 Outil: {step['tool']}")
    if step['dependencies']:
        print(f"   🔗 Dépend de: {step['dependencies']}")

# Affichage du résultat final
print("\n" + "="*60)
print("\n📝 RÉSULTAT FINAL:")
display(Markdown(result['final_result']))

### 4.2 Visualisation de l'Exécution du Plan

In [ ]:
# Fonction pour visualiser l'exécution du plan
def visualize_plan_execution(execution_history):
    """Crée une visualisation HTML de l'exécution du plan"""
    
    html = """
    <div style="font-family: Arial, sans-serif;">
        <h3>🔄 Historique d'Exécution</h3>
        <div style="margin-left: 20px;">
    """
    
    for entry in execution_history:
        html += f"""
        <div style="margin: 10px 0; padding: 10px; background: #f0f0f0; border-left: 3px solid #4CAF50;">
            <strong>Étape {entry['step_id']}</strong>: {entry['description']}<br>
            {f'<em>Outil: {entry["tool"]}</em><br>' if entry.get('tool') else ''}
            <div style="margin-top: 5px; color: #666;">
                Résultat: {entry.get('result', 'N/A')[:200]}...
            </div>
        </div>
        """
    
    html += """
        </div>
    </div>
    """
    
    return HTML(html)

# Visualisation de l'historique
if result.get('execution_history'):
    display(visualize_plan_execution(result['execution_history']))

## Chapitre 5: Cas Pratiques Avancés

### 5.1 Création d'un Agent Spécialisé pour l'Analyse de Données

In [ ]:
# Agent spécialisé pour l'analyse de données
from langchain_core.tools import tool

@tool
def analyze_sales_trend() -> str:
    """Analyse les tendances de ventes dans la base de données"""
    try:
        # Connexion à la base de données
        from rag_attack.utils.config import get_sql_connection
        import pandas as pd
        
        conn = get_sql_connection()
        
        # Requête pour analyser les tendances
        query = """
        SELECT 
            YEAR(order_date) as year,
            MONTH(order_date) as month,
            COUNT(*) as num_orders,
            SUM(total_amount) as total_sales
        FROM orders
        GROUP BY YEAR(order_date), MONTH(order_date)
        ORDER BY year DESC, month DESC
        """
        
        df = pd.read_sql(query, conn)
        conn.close()
        
        # Analyse simple
        if len(df) > 0:
            avg_sales = df['total_sales'].mean()
            trend = "croissante" if df['total_sales'].iloc[0] > avg_sales else "décroissante"
            
            return f"""
            Analyse des tendances de ventes:
            - Nombre total de mois analysés: {len(df)}
            - Ventes moyennes mensuelles: {avg_sales:.2f}€
            - Tendance actuelle: {trend}
            - Dernier mois: {df['num_orders'].iloc[0]} commandes pour {df['total_sales'].iloc[0]:.2f}€
            """
        else:
            return "Aucune donnée de vente disponible"
            
    except Exception as e:
        return f"Erreur lors de l'analyse: {str(e)}"

# Création de l'agent analyste
data_analyst_agent = SimpleToolAgent(
    tools=[
        analyze_sales_trend,
        sql_query_tool,
        azure_search_tool
    ],
    system_prompt="""Tu es un analyste de données expert pour VéloCorp.
    Ton rôle est d'analyser les données de ventes et de fournir des insights business.
    Utilise les outils disponibles pour extraire et analyser les données pertinentes."""
)

print("✅ Agent analyste de données créé")

In [ ]:
# Test de l'agent analyste
analysis_question = "Analyse les performances de ventes et identifie les opportunités d'amélioration"

print(f"📊 Question d'analyse: {analysis_question}\n")
print("="*60)

analyst_response = data_analyst_agent.invoke(analysis_question)
display(Markdown(analyst_response))

### 5.2 Agent avec Mémoire Conversationnelle

In [ ]:
# Implémentation d'un agent avec mémoire
from typing import List
from langchain_core.messages import HumanMessage, AIMessage

class ConversationalAgent:
    """Agent qui maintient l'historique de la conversation"""
    
    def __init__(self, tools, max_history=5):
        self.agent = SimpleToolAgent(tools=tools)
        self.history = []
        self.max_history = max_history
    
    def chat(self, user_input: str) -> str:
        # Construire le contexte avec l'historique
        context = "\n".join([
            f"User: {h['user']}\nAssistant: {h['assistant']}"
            for h in self.history[-self.max_history:]
        ])
        
        # Ajouter le contexte à la question si historique existe
        if context:
            full_input = f"""Contexte de la conversation:
{context}

Nouvelle question: {user_input}"""
        else:
            full_input = user_input
        
        # Obtenir la réponse
        response = self.agent.invoke(full_input)
        
        # Sauvegarder dans l'historique
        self.history.append({
            "user": user_input,
            "assistant": response
        })
        
        return response
    
    def reset(self):
        """Réinitialise l'historique de conversation"""
        self.history = []

# Création de l'agent conversationnel
conversational_agent = ConversationalAgent(
    tools=[azure_search_tool, sql_query_tool]
)

print("✅ Agent conversationnel créé avec mémoire")

In [ ]:
# Simulation d'une conversation
conversation = [
    "Quels sont les modèles de vélos électriques de VéloCorp ?",
    "Quel est le prix du modèle le plus cher ?",
    "Y a-t-il des commandes récentes pour ce modèle ?"
]

print("💬 CONVERSATION AVEC MÉMOIRE\n")
print("="*60)

for i, question in enumerate(conversation, 1):
    print(f"\n👤 Question {i}: {question}")
    print("-" * 40)
    
    response = conversational_agent.chat(question)
    print(f"🤖 Réponse: {response[:300]}..." if len(response) > 300 else f"🤖 Réponse: {response}")

print("\n" + "="*60)
print("\n💡 Note: L'agent se souvient du contexte des questions précédentes !")

## Chapitre 6: Bonnes Pratiques et Optimisations

### 6.1 Gestion des Erreurs et Fallback

In [ ]:
# Agent robuste avec gestion d'erreurs
from langchain_core.tools import tool
import time

@tool
def safe_search_with_retry(query: str, max_retries: int = 3) -> str:
    """Recherche avec gestion d'erreurs et retry automatique"""
    
    for attempt in range(max_retries):
        try:
            # Tentative de recherche
            result = azure_search_tool.invoke(query)
            
            if result and "No results" not in result:
                return result
            
            # Si pas de résultats, essayer avec une requête modifiée
            if attempt < max_retries - 1:
                # Simplifier la requête
                words = query.split()
                if len(words) > 2:
                    query = " ".join(words[:2])  # Garder seulement les 2 premiers mots
                time.sleep(1)  # Petit délai entre les tentatives
                
        except Exception as e:
            if attempt == max_retries - 1:
                return f"Erreur après {max_retries} tentatives: {str(e)}"
            time.sleep(2 ** attempt)  # Backoff exponentiel
    
    return "Aucun résultat trouvé après plusieurs tentatives"

# Test de la recherche robuste
test_queries = [
    "vélos électriques premium haute performance 2024",  # Requête complexe
    "xyzabc123nonexistent",  # Requête qui ne donnera pas de résultats
]

print("🛡️ TEST DE ROBUSTESSE\n")
print("="*60)

for query in test_queries:
    print(f"\n🔍 Recherche: {query}")
    result = safe_search_with_retry(query)
    print(f"📝 Résultat: {result[:200]}..." if len(result) > 200 else f"📝 Résultat: {result}")

### 6.2 Monitoring et Métriques de Performance

In [ ]:
# Système de monitoring pour les agents
import time
from datetime import datetime
from typing import Dict, Any

class AgentMonitor:
    """Monitore les performances des agents"""
    
    def __init__(self):
        self.metrics = []
    
    def track_execution(self, agent_name: str, question: str, agent_func):
        """Track l'exécution d'un agent"""
        start_time = time.time()
        
        try:
            result = agent_func(question)
            success = True
            error = None
        except Exception as e:
            result = None
            success = False
            error = str(e)
        
        execution_time = time.time() - start_time
        
        # Enregistrer les métriques
        self.metrics.append({
            "timestamp": datetime.now(),
            "agent": agent_name,
            "question_length": len(question),
            "execution_time": execution_time,
            "success": success,
            "error": error,
            "response_length": len(result) if result else 0
        })
        
        return result
    
    def get_statistics(self) -> Dict[str, Any]:
        """Calcule les statistiques de performance"""
        if not self.metrics:
            return {"message": "Aucune métrique disponible"}
        
        import pandas as pd
        df = pd.DataFrame(self.metrics)
        
        stats = {
            "total_executions": len(df),
            "success_rate": (df['success'].sum() / len(df)) * 100,
            "avg_execution_time": df['execution_time'].mean(),
            "max_execution_time": df['execution_time'].max(),
            "min_execution_time": df['execution_time'].min(),
            "avg_response_length": df['response_length'].mean()
        }
        
        # Statistiques par agent
        agent_stats = {}
        for agent in df['agent'].unique():
            agent_df = df[df['agent'] == agent]
            agent_stats[agent] = {
                "executions": len(agent_df),
                "avg_time": agent_df['execution_time'].mean(),
                "success_rate": (agent_df['success'].sum() / len(agent_df)) * 100
            }
        
        stats["by_agent"] = agent_stats
        
        return stats

# Test du monitoring
monitor = AgentMonitor()

# Exécuter plusieurs requêtes avec monitoring
test_questions = [
    "Quels sont les vélos électriques ?",
    "Analyse les ventes du dernier trimestre",
    "Trouve les opportunités CRM importantes"
]

print("📊 MONITORING DES AGENTS\n")
print("="*60)

for q in test_questions:
    print(f"\n⏱️ Exécution: {q[:50]}...")
    
    # Test avec agent simple
    result = monitor.track_execution(
        "SimpleAgent",
        q,
        lambda x: simple_agent.invoke(x)
    )
    
    # Test avec agent ReAct
    result = monitor.track_execution(
        "ReActAgent",
        q,
        lambda x: react_agent.invoke(x)
    )

# Afficher les statistiques
stats = monitor.get_statistics()

print("\n" + "="*60)
print("\n📈 STATISTIQUES DE PERFORMANCE:\n")
print(f"Total d'exécutions: {stats['total_executions']}")
print(f"Taux de succès: {stats['success_rate']:.1f}%")
print(f"Temps moyen: {stats['avg_execution_time']:.2f}s")
print(f"Temps min/max: {stats['min_execution_time']:.2f}s / {stats['max_execution_time']:.2f}s")

print("\n📊 Par Agent:")
for agent, agent_stats in stats['by_agent'].items():
    print(f"\n  {agent}:")
    print(f"    - Exécutions: {agent_stats['executions']}")
    print(f"    - Temps moyen: {agent_stats['avg_time']:.2f}s")
    print(f"    - Taux de succès: {agent_stats['success_rate']:.1f}%")

## Conclusion et Ressources

### 🎯 Ce que nous avons appris

1. **Agents Simples** : Comment créer des agents qui utilisent des outils pour répondre aux questions
2. **Framework ReAct** : L'importance de la transparence dans le raisonnement
3. **Multi-Outils** : Orchestration de plusieurs sources de données
4. **Planification** : Décomposition de tâches complexes
5. **Robustesse** : Gestion d'erreurs et monitoring

### 🚀 Prochaines Étapes

1. **Expérimenter** avec différentes combinaisons d'outils
2. **Personnaliser** les prompts système pour votre domaine
3. **Intégrer** ces agents dans vos applications
4. **Optimiser** les performances avec du caching et de la parallélisation

### 📚 Ressources Additionnelles

- [Documentation LangGraph](https://python.langchain.com/docs/langgraph)
- [Documentation LangChain](https://python.langchain.com/docs/get_started/introduction)
- [Azure Cognitive Search](https://docs.microsoft.com/azure/search/)
- [Papers sur ReAct](https://arxiv.org/abs/2210.03629)

### 💡 Points Clés à Retenir

1. **Les agents transforment le RAG** de passif à actif
2. **La transparence** du raisonnement est cruciale
3. **L'orchestration** d'outils multiplie les capacités
4. **La planification** permet de résoudre des problèmes complexes
5. **Le monitoring** est essentiel en production

## Exercices Pratiques

### Exercice 1: Créer votre propre outil

Créez un outil personnalisé qui calcule des métriques business spécifiques.

In [ ]:
# TODO: Créez votre propre outil ici
# Exemple: Un outil qui calcule le ROI d'une campagne marketing

@tool
def calculate_roi(investment: float, revenue: float) -> str:
    """Calcule le ROI (Return on Investment)
    
    Args:
        investment: Montant investi
        revenue: Revenue généré
    
    Returns:
        ROI en pourcentage
    """
    # Votre implémentation ici
    pass

### Exercice 2: Agent Spécialisé

Créez un agent spécialisé pour un cas d'usage spécifique de votre choix.

In [ ]:
# TODO: Créez votre agent spécialisé
# Exemple: Un agent pour le support client

# Votre code ici

### Exercice 3: Plan Complexe

Utilisez le planner pour résoudre un problème business réel de votre organisation.

In [ ]:
# TODO: Définissez un objectif complexe et utilisez le planner
# Exemple: "Préparer le lancement d'un nouveau produit"

# Votre code ici

---

🎉 **Félicitations !** Vous maîtrisez maintenant les concepts avancés du RAG agentique avec LangGraph !

N'hésitez pas à expérimenter et adapter ces exemples à vos besoins spécifiques.